In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from tslearn.metrics import cdist_dtw, dtw
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesResampler


In [ ]:
def load_tracking_data():
    tracking_url = "https://raw.githubusercontent.com/nfl-football-ops/Big-Data-Bowl/master/Data/tracking_gameId_2017090700.csv"
    return pd.read_csv(tracking_url)

def load_pass_plays():
    plays_url = "https://raw.githubusercontent.com/nfl-football-ops/Big-Data-Bowl/master/Data/plays.csv"
    plays_df = pd.read_csv(plays_url)
    return plays_df[plays_df['PassResult'].notnull()]

def load_players():
    url = "https://raw.githubusercontent.com/nfl-football-ops/Big-Data-Bowl/master/Data/players.csv"
    return pd.read_csv(url).rename(columns={'nflId': 'player_id'})

In [ ]:
def create_display_names(players_df):
    players_df = players_df.copy()
    players_df['displayName'] = players_df['FirstName'].str.strip() + ' ' + players_df['LastName'].str.strip()
    return players_df

def get_receivers_df(players_df, positions=['WR', 'TE', 'RB']):
    return players_df[players_df['PositionAbbr'].isin(positions)].copy()

def get_tracking_receivers(tracking_df, receivers_df):
    return tracking_df.merge(receivers_df, on='displayName', how='inner')

def get_snap_frames(tracking_receivers):
    snap = tracking_receivers[tracking_receivers['event'] == 'ball_snap']
    snap_frames = snap.groupby(['gameId', 'playId'])['frame.id'].min().reset_index()
    return snap_frames.rename(columns={'frame.id': 'snap_frame'})

def filter_post_snap(tracking_receivers, snap_frames):
    merged = tracking_receivers.merge(snap_frames, on=['gameId', 'playId'], how='left')
    # Ensure standard naming for frame columns
    merged = merged.rename(columns={'frame.id': 'frameId'})
    return merged[merged['frameId'] >= merged['snap_frame']]


In [ ]:
def clean_route(route, min_length=5):
    route = route.dropna(subset=['x', 'y'])
    path = np.array(list(zip(route['x'], route['y'])))
    if np.all(np.isfinite(path)) and len(path) > min_length:
        return path
    return None

def extract_clean_routes(df):
    grouped = df.groupby(['displayName', 'playId'])
    cleaned_paths = []
    route_keys = []

    for (name, pid), route in grouped:
        clean = clean_route(route)
        if clean is not None:
            cleaned_paths.append(clean)
            route_keys.append((name, pid))

    return cleaned_paths, route_keys


In [ ]:
def clean_route(route, min_length=5):
    route = route.dropna(subset=['x', 'y'])
    path = np.array(list(zip(route['x'], route['y'])))
    if np.all(np.isfinite(path)) and len(path) > min_length:
        return path
    return None

def extract_clean_routes(df):
    grouped = df.groupby(['displayName', 'playId'])
    cleaned_paths = []
    route_keys = []

    for (name, pid), route in grouped:
        clean = clean_route(route)
        if clean is not None:
            cleaned_paths.append(clean)
            route_keys.append((name, pid))

    return cleaned_paths, route_keys


In [ ]:
def preprocess_routes(route_paths, route_keys, target_len=50, return_raw=False):
    filtered_routes = []
    filtered_keys = []

    for r, key in zip(route_paths, route_keys):
        if len(r) >= target_len:
            filtered_routes.append(r)
            filtered_keys.append(key)

    resampled = TimeSeriesResampler(sz=target_len).fit_transform(filtered_routes)
    scaler = StandardScaler()
    normalized = np.array([scaler.fit_transform(r) for r in resampled])

    if return_raw:
        return normalized, filtered_keys, filtered_routes
    else:
        return normalized, filtered_keys


In [ ]:
def average_trajectory(cluster_routes):
    max_len = max(len(r) for r in cluster_routes)
    padded = np.array([
        np.pad(r, ((0, max_len - len(r)), (0, 0)), constant_values=np.nan)
        for r in cluster_routes
    ])
    return np.nanmean(padded, axis=0)

def plot_clusters_with_average(routes_scaled, labels):
    clustered = defaultdict(list)
    for r, label in zip(routes_scaled, labels):
        clustered[label].append(r)

    for label, group in clustered.items():
        avg = average_trajectory(group)
        fig, ax = plt.subplots(figsize=(6, 6))
        for r in group:
            ax.plot(r[:, 0], r[:, 1], alpha=0.3, color='gray')
        ax.plot(avg[:, 0], avg[:, 1], color='red', linewidth=2, label='Average')
        ax.set_title(f"Cluster {label} – Average Route")
        ax.legend()
        ax.grid(True)
        save_path = f"cluster_visualizations/cluster_{label}_average_route.png"
        fig.savefig(save_path)
        plt.close(fig)


def plot_centroids(model):
    fig, ax = plt.subplots(figsize=(6, 6))
    for i, centroid in enumerate(model.cluster_centers_):
        ax.scatter(centroid[0], centroid[1], marker='X', s=200, label=f'Cluster {i}')
    ax.set_title("Cluster Centroids")
    ax.legend()
    save_path = "cluster_visualizations/cluster_centroids.png"
    fig.savefig(save_path)
    plt.close(fig)


def plot_cluster_distribution(labels):
    fig, ax = plt.subplots(figsize=(6, 6))
    counts = Counter(labels)
    sns.barplot(x=list(counts.keys()), y=list(counts.values()), ax=ax)
    ax.set_title("Cluster Size Distribution")
    ax.set_xlabel("Cluster")
    ax.set_ylabel("Number of Routes")
    save_path = "cluster_visualizations/cluster_distribution.png"
    fig.savefig(save_path)
    plt.close(fig)


In [ ]:
def dtw_distance(ts_a, ts_b):
    M, N = len(ts_a), len(ts_b)
    dtw_matrix = np.full((M + 1, N + 1), np.inf)
    dtw_matrix[0, 0] = 0
    for i in range(1, M + 1):
        for j in range(1, N + 1):
            cost = abs(ts_a[i - 1] - ts_b[j - 1])
            last_min = min(
                dtw_matrix[i - 1, j],
                dtw_matrix[i, j - 1],
                dtw_matrix[i - 1, j - 1]
            )
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix[M, N]

In [ ]:
def compute_total_distance(route):
    dx = np.diff(route[:, 0])
    dy = np.diff(route[:, 1])
    return np.sum(np.sqrt(dx**2 + dy**2))

def compute_straightness(route):
    start, end = route[0], route[-1]
    displacement = np.linalg.norm(end - start)
    length = compute_total_distance(route)
    return displacement / length if length != 0 else 0

def compute_curvature_density(route):
    angles = []
    for i in range(1, len(route) - 1):
        vec1 = route[i] - route[i - 1]
        vec2 = route[i + 1] - route[i]
        norm_product = np.linalg.norm(vec1) * np.linalg.norm(vec2)
        if norm_product == 0:
            continue
        angle = np.arccos(np.clip(np.dot(vec1, vec2) / norm_product, -1.0, 1.0))
        angles.append(angle)
    total_length = compute_total_distance(route)
    return np.mean(angles) / total_length if total_length != 0 else 0


In [ ]:
def save_boxplots_by_metric(df_routes, metrics):
    os.makedirs("cluster_visualizations", exist_ok=True)
    fig, axs = plt.subplots(1, len(metrics), figsize=(22, 5))

    for i, metric in enumerate(metrics):
        df_routes.boxplot(column=metric, by="Route Type", ax=axs[i])
        axs[i].set_title(f"{metric} Distribution by Route Type")
        axs[i].tick_params(axis='x', rotation=45)
        axs[i].set_xlabel("Route Type")
        axs[i].set_ylabel(metric)

    plt.tight_layout()
    plt.suptitle("")
    save_path = "cluster_visualizations/boxplot_metrics_by_route_type.png"
    fig.savefig(save_path)
    plt.close(fig)


In [ ]:
def save_barplots_by_metric(avg_stats, metrics, colors):
    os.makedirs("cluster_visualizations", exist_ok=True)
    fig, axs = plt.subplots(1, len(metrics), figsize=(22, 5))

    for i, metric in enumerate(metrics):
        axs[i].bar(avg_stats["Route Type"], avg_stats[metric], color=colors[i])
        axs[i].set_title(f"Average {metric} by Route Type")
        axs[i].tick_params(axis='x', rotation=45)
        axs[i].set_ylabel(metric)

    plt.tight_layout()
    save_path = "cluster_visualizations/barplot_avg_metrics_by_route_type.png"
    fig.savefig(save_path)
    plt.close(fig)
